In [1]:
from jenga.tasks.openml import OpenMLBinaryClassificationTask, OpenMLMultiClassClassificationTask
from jenga.corruptions.generic import MissingValues
    
import pandas as pd
import numpy as np

from data_imputation_paper.imputation.simple import ModeImputer
from data_imputation_paper.imputation.ml import KNNImputer, ForestImputer
from data_imputation_paper.imputation.dl import AutoKerasImputer
from data_imputation_paper.imputation.generative import GAINImputer
from data_imputation_paper.evaluation import Evaluator

## Make Deterministic

In [2]:
seed = 42

## Create example tasks

In [3]:
task = OpenMLMultiClassClassificationTask(openml_id=4552, seed=seed)

## Insert missing values using jenga

In [4]:
missing_values = [
    MissingValues(column='V2', fraction=0.5, na_value=np.nan, missingness='MCAR'),
    MissingValues(column='V4', fraction=0.5, na_value=np.nan, missingness='MCAR'),
    MissingValues(column='V15', fraction=0.5, na_value=np.nan, missingness='MCAR')
]

## Mode Imputation

In [5]:
arguments = {
    "seed": seed
}

%time Evaluator(task, missing_values, ModeImputer, arguments).evaluate(10).report()

Evaluation result contains 3 target columns: V2, V4, V15
All are in a round-robin fashion imputed and performances are as follows:

Target Column: V2
            train         test
MAE     30.041142    28.867716
MSE   1444.706256  1245.957937
RMSE    38.008059    35.292878

Target Column: V4
                train      test
F1_micro     0.828199  0.841696
F1_macro     0.453008  0.457008
F1_weighted  0.750384  0.769375

Target Column: V15
                train      test
F1_micro     0.121756  0.112014
F1_macro     0.013566  0.012749
F1_weighted  0.026462  0.022708

CPU times: user 2.2 s, sys: 24.2 ms, total: 2.22 s
Wall time: 2.22 s


## KNN imputation

In [6]:
arguments = {
    "seed": seed,
    "hyperparameter_grid_categorical_imputer": {
        "n_neighbors": [3, 5]
    },
    "hyperparameter_grid_numerical_imputer": {
        "n_neighbors": [3, 5]
    }
}

%time Evaluator(task, missing_values, KNNImputer, arguments).evaluate(5).report()

Evaluation result contains 3 target columns: V2, V4, V15
All are in a round-robin fashion imputed and performances are as follows:

Target Column: V2
            train         test
MAE     31.184731    29.788198
MSE   1549.078627  1389.964961
RMSE    39.356120    37.282137

Target Column: V4
                train      test
F1_micro     0.935922  0.942403
F1_macro     0.881156  0.881266
F1_weighted  0.934604  0.941230

Target Column: V15
                train      test
F1_micro     0.510591  0.525442
F1_macro     0.417909  0.414359
F1_weighted  0.503308  0.514905

CPU times: user 11 s, sys: 2.72 s, total: 13.8 s
Wall time: 12 s


## Forest imputation

In [7]:
arguments = {
    "seed": seed,
    "hyperparameter_grid_categorical_imputer": {
        "n_estimators": [50, 100]
    },
    "hyperparameter_grid_numerical_imputer": {
        "n_estimators": [50, 100]
    }
}

%time Evaluator(task, missing_values, ForestImputer, arguments).evaluate(5).report()

Evaluation result contains 3 target columns: V2, V4, V15
All are in a round-robin fashion imputed and performances are as follows:

Target Column: V2
            train         test
MAE     28.215426    26.842464
MSE   1276.326227  1138.658181
RMSE    35.720128    33.736769

Target Column: V4
                train      test
F1_micro     0.953045  0.955124
F1_macro     0.911836  0.911809
F1_weighted  0.951996  0.954757

Target Column: V15
                train      test
F1_micro     0.563195  0.553710
F1_macro     0.475594  0.436930
F1_weighted  0.558138  0.547215

CPU times: user 43.6 s, sys: 1.2 s, total: 44.8 s
Wall time: 32.1 s


## AutoKeras imputation

In [ ]:
arguments = {
    "seed": seed,
    'max_trials': 2,
    'tuner': 'greedy',
    'validation_split': 0.1,
    'epochs': 2
}

%time Evaluator(task, missing_values, AutoKerasImputer, arguments).evaluate(5).report()

## GAIN imputation

In [5]:
arguments = {
    "seed": seed,
    "hyperparameter_grid": {
        "gain": {
            "alpha": [80, 120],
            "hint_rate": [0.5, 0.9],
            "noise": [0.001, 0.1]
        },
        "training": {
            "batch_size": [64, 256],
            "epochs": [5, 15]
        }
    }
}

%time Evaluator(task, missing_values, GAINImputer, arguments).evaluate(2).report()

Evaluation result contains 3 target columns: V2, V4, V15
All are in a round-robin fashion imputed and performances are as follows:

Target Column: V2
            train         test
MAE     31.764334    30.070516
MSE   1700.280506  1480.509226
RMSE    41.161238    38.465811

Target Column: V4
                train      test
F1_micro     0.831642  0.849823
F1_macro     0.454039  0.459400
F1_weighted  0.755206  0.780847

Target Column: V15
                train      test
F1_micro     0.042807  0.038869
F1_macro     0.019164  0.016890
F1_weighted  0.031243  0.028484

CPU times: user 15.5 s, sys: 1.06 s, total: 16.6 s
Wall time: 14.9 s
